<a href="https://colab.research.google.com/github/Factorito/AIportplio/blob/main/Termproject_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#install pip(필수)



In [ ]:
# 1. numpy, transformers, datasets 완전히 제거
!pip uninstall -y numpy transformers datasets

# 2. 안정 버전 명시적으로 재설치
!pip install numpy==1.26.4 transformers==4.38.2 datasets==2.18.0 accelerate==0.34.2

# 3. 강제 런타임 재시작, 이 셀 실행후 런타임 끊어지면 아래 셀부터 실행해주시면 됩니다. 감사합니다.
import os
os.kill(os.getpid(), 9)


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: datasets 2.14.4
Uninstalling datasets-2.14.4:
  Successfully uninstalled datasets-2.14.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 477.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
!pip install -U gdown
!pip install -U transformers==4.38.2
!pip install tensorboard
!pip install -U timm
!pip install -U peft==0.8.2

In [ ]:
import gdown
import zipfile
import os

# 파일 ID 입력
file_id = "1m4sgrQ8xTGl17IIABtfeZm89AgGt_SZI"

# 다운로드 받을 파일 이름 지정
url = f"https://drive.google.com/uc?id={file_id}"

output = "train.zip"  # 원하는 이름으로 변경 가능

# 다운로드 수행
gdown.download(url, output, quiet=False)

# 결과 파일 저장 경로
os.makedirs("release", exist_ok=True)

# 압축 해제할 경로
extract_dir = "data"
os.makedirs(extract_dir, exist_ok=True)

# 압축 풀기
with zipfile.ZipFile("train.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("압축 해제 완료:", extract_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1m4sgrQ8xTGl17IIABtfeZm89AgGt_SZI
From (redirected): https://drive.google.com/uc?id=1m4sgrQ8xTGl17IIABtfeZm89AgGt_SZI&confirm=t&uuid=bc5e9970-7b46-4802-a678-1c0a5366a646
To: /content/train.zip
100%|██████████| 3.11G/3.11G [00:31<00:00, 99.6MB/s]


압축 해제 완료: data


In [ ]:
import numpy as np
import torch
from transformers import AutoFeatureExtractor
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

# Train code(테스트 시 스킵 해주세요)


In [ ]:
from datasets import Dataset
import pandas as pd
import os
from PIL import Image
from transformers import AutoFeatureExtractor
import torch
import numpy as np # Import numpy

# 이미지 유효성 검사
def is_valid_image(path):
    try:
        with Image.open(path) as img:
            img.verify()
        return True
    except Exception:
        return False

# CSV 로드 및 이미지 경로 설정
df = pd.read_csv("./data/train/train_labels.csv")
df["image"] = df["file_name"].apply(lambda x: os.path.join("./data/train/images", x))
df = df[df["image"].apply(is_valid_image)].reset_index(drop=True)

# HuggingFace Dataset으로 변환
dataset = Dataset.from_pandas(df)

# Feature extractor 로드
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# 이미지 전처리
def preprocess_images(examples):
    images = [Image.open(path).convert("RGB") for path in examples["image"]]
    inputs = feature_extractor(images=images, return_tensors="np")
    examples["pixel_values"] = torch.tensor(inputs["pixel_values"], dtype=torch.float32)
    return examples

# 라벨 묶기 및 유효성 검사
def group_labels(example, num_styles, num_fruits):
    style_label = int(example["style"])
    fruit_label = int(example["fruit"])


    assert 0 <= style_label < num_styles, f"Style label {style_label} is out of expected range [0, {num_styles-1}]"
    assert 0 <= fruit_label < num_fruits, f"Fruit label {fruit_label} is out of expected range [0, {num_fruits-1}]"

    return {
        "labels": (style_label, fruit_label)
    }


num_styles = df['style'].nunique()
num_fruits = df['fruit'].nunique()

# 전체 전처리 적용
dataset = dataset.map(preprocess_images, batched=True)
dataset = dataset.map(lambda example: group_labels(example, num_styles, num_fruits))

# 불필요한 열 제거
dataset = dataset.remove_columns(["image", "file_name", "style", "fruit"])

# Train/Val 분리
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

# 토치 포맷
train_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])
val_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Map:   0%|          | 0/6379 [00:00<?, ? examples/s]

Map:   0%|          | 0/6379 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset)
print(val_dataset)
print(train_dataset[1])
#dataset 검사

Dataset({
    features: ['pixel_values', 'labels'],
    num_rows: 5103
})
Dataset({
    features: ['pixel_values', 'labels'],
    num_rows: 1276
})
{'pixel_values': tensor([[[ 0.4902,  0.4745,  0.4510,  ...,  0.2784,  0.2706,  0.2863],
         [ 0.5059,  0.4902,  0.4902,  ...,  0.3098,  0.2941,  0.3255],
         [ 0.5216,  0.5216,  0.5294,  ...,  0.3098,  0.3490,  0.3804],
         ...,
         [ 0.5529,  0.5216,  0.4824,  ...,  0.1922,  0.2078,  0.2157],
         [ 0.5451,  0.5373,  0.5373,  ...,  0.2784,  0.2863,  0.2549],
         [ 0.5216,  0.5216,  0.5686,  ...,  0.3020,  0.3020,  0.2784]],

        [[ 0.4196,  0.3961,  0.3725,  ...,  0.1843,  0.1608,  0.1686],
         [ 0.4431,  0.4039,  0.4039,  ...,  0.2078,  0.1843,  0.2000],
         [ 0.4510,  0.4353,  0.4353,  ...,  0.2000,  0.2392,  0.2627],
         ...,
         [ 0.4353,  0.4039,  0.3725,  ...,  0.0980,  0.1059,  0.1137],
         [ 0.4431,  0.4275,  0.4196,  ...,  0.1686,  0.1843,  0.1608],
         [ 0.4118,  0.40

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import Trainer
import torch.nn as nn
import torch

class MultiTaskTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels_style, labels_fruit = inputs.pop("labels")

        outputs = model(**inputs)
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(outputs["style"], labels_style) + loss_fct(outputs["fruit"], labels_fruit)

        return (loss, outputs) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):

        labels_style, labels_fruit = inputs.pop("labels")

        with torch.no_grad():
            outputs = model(**inputs)

        style_logits = outputs["style"]
        fruit_logits = outputs["fruit"]

        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(style_logits, labels_style) + loss_fct(fruit_logits, labels_fruit)

        if prediction_loss_only:
            return (loss, None, None)


        return (
           loss,
           (style_logits.detach().cpu(), fruit_logits.detach().cpu()),
           (labels_style.detach().cpu(), labels_fruit.detach().cpu()),
        )

def compute_metrics(pred):
    logits_tuple = pred.predictions
    labels_tuple = pred.label_ids


    style_preds = np.argmax(logits_tuple[0], axis=1)
    fruit_preds = np.argmax(logits_tuple[1], axis=1)


    style_labels = labels_tuple[0]
    fruit_labels = labels_tuple[1]

    return {
        "eval_style_acc": accuracy_score(style_labels, style_preds),
        "eval_fruit_acc": accuracy_score(fruit_labels, fruit_preds),
    }

In [ ]:
def custom_collate_fn(batch):

    pixel_values = torch.stack([example["pixel_values"].squeeze(0) for example in batch])

    style_labels = torch.tensor([example["labels"][0] for example in batch], dtype=torch.long)
    fruit_labels = torch.tensor([example["labels"][1] for example in batch], dtype=torch.long)

    global num_styles, num_fruits
    assert torch.all(style_labels >= 0) and torch.all(style_labels < num_styles), \
        f"Style labels out of range. Expected [0, {num_styles-1}], got min {style_labels.min().item()}, max {style_labels.max().item()}"
    assert torch.all(fruit_labels >= 0) and torch.all(fruit_labels < num_fruits), \
        f"Fruit labels out of range. Expected [0, {num_fruits-1}], got min {fruit_labels.min().item()}, max {fruit_labels.max().item()}"


    return {
          "pixel_values": pixel_values,
        "labels": (style_labels, fruit_labels),
    }

In [ ]:
from transformers import TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    remove_unused_columns=False,
    #fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_style_acc",
    greater_is_better=True,
    report_to="none",
)

# 모델 업데이트
model = ViTMultiTask(num_styles=num_styles, num_fruits=num_fruits)


trainer = MultiTaskTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=custom_collate_fn,
)



trainer.train()

torch.save(model, "model.pt") # 모델 저장

predictions = trainer.predict(val_dataset)
print(predictions.metrics)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Style Acc,Fruit Acc
1,1.665100,0.497713,0.998433,0.996082
2,0.344100,0.294380,0.999216,0.997649
3,0.266300,0.229025,0.998433,0.998433
4,0.202400,0.200207,1.000000,0.998433
5,0.182600,0.191973,1.000000,0.998433


{'test_loss': 0.20020699501037598, 'test_eval_style_acc': 1.0, 'test_eval_fruit_acc': 0.9984326018808778, 'test_runtime': 28.6083, 'test_samples_per_second': 44.602, 'test_steps_per_second': 0.699}


#Test Code

In [ ]:
import torch
import torch.nn as nn
from transformers import ViTModel

class ViTMultiTask(nn.Module):
    def __init__(self, model_name='google/vit-base-patch16-224-in21k', num_styles=3, num_fruits=5):
        super().__init__()
        self.vit = ViTModel.from_pretrained(model_name)
        hidden_size = self.vit.config.hidden_size
        self.style_head = nn.Linear(hidden_size, num_styles)
        self.fruit_head = nn.Linear(hidden_size, num_fruits)

    def forward(self, pixel_values):
        outputs = self.vit(pixel_values=pixel_values)
        cls_output = outputs.last_hidden_state[:, 0, :]  # CLS 토큰
        return {
            "style": self.style_head(cls_output),  # shape: (batch_size, num_styles)
            "fruit": self.fruit_head(cls_output)   # shape: (batch_size, num_fruits)
        }


In [ ]:
#모델 업데이트
import gdown
import zipfile
import os
import torch

# 파일 ID재입력

file_id = "1eYLJ0Fw8oI_a7GE8D_nSdXVeWu4GVLzA"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

output = "model.pt"  # 원하는 이름으로 변경 가능

# 다운로드 수행
gdown.download(url, output, quiet=False)

# 모델 저장 경로
os.makedirs("data", exist_ok=True)

print("모델 다운로드 완료 완료:")

# 모델 로딩
saved_model = torch.load("model.pt", weights_only=False)
saved_model

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1eYLJ0Fw8oI_a7GE8D_nSdXVeWu4GVLzA
From (redirected): https://drive.google.com/uc?export=download&id=1eYLJ0Fw8oI_a7GE8D_nSdXVeWu4GVLzA&confirm=t&uuid=3f6e628b-e13d-4b75-a65d-77ff8c0f9736
To: /content/model.pt
100%|██████████| 346M/346M [00:03<00:00, 111MB/s]


모델 다운로드 완료 완료:


ViTMultiTask(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_featu

In [ ]:
import torch
import re
import os
from tqdm import tqdm
from datasets import Dataset
from PIL import Image
from datasets import load_dataset
import pandas as pd
from transformers import AutoFeatureExtractor

#  다운로드한 모델을 평가 모드로 전환
saved_model.eval()

# Move the model to the appropriate device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
saved_model.to(device)


# 저장 경로 설정
output_dir = './release'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'test.task1.txt')

# 결과 저장 리스트
results = []

# 훈련 데이터 로딩 시 사용했던 DataFrame 재사용 또는 새로 로드
try:
    df
except NameError:

    def is_valid_image(path):
        try:
            with Image.open(path) as img:
                img.verify()
            return True
        except Exception:
            return False

    df = pd.read_csv("./data/test/test_labels.csv") #여기를 ./data/test/test_labes.csv로 커스텀 해주세요
    df["image"] = df["file_name"].apply(lambda x: os.path.join("./data/test/images", x)) #여기를 ./data/test/images로 커스텀 해주세요
    df = df[df["image"].apply(is_valid_image)].reset_index(drop=True)


#허깅페이스에 맞게 변환
inference_dataset = Dataset.from_pandas(df)
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# 예측 수행
for i, example in enumerate(tqdm(inference_dataset)):
    image_path = example["image"]
    file_name = example["file_name"]

    image = Image.open(image_path).convert("RGB")

    inputs = feature_extractor(images=image, return_tensors="pt").to(saved_model.vit.device)


    with torch.no_grad():
        outputs = saved_model(**inputs)

        style_pred = torch.argmax(outputs["style"], dim=1).item()
        fruit_pred = torch.argmax(outputs["fruit"], dim=1).item()


        image_id = int(re.sub(r"\D", "", file_name))

        # 탭 구분으로 결과 저장
        results.append(f"{image_id}\t{style_pred}\t{fruit_pred}\n")

# 결과 파일로 저장
with open(output_path, 'w') as f:
    f.writelines(results)

print(f"✅ 완료: {output_path}에 결과 저장됨.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
100%|██████████| 6379/6379 [03:27<00:00, 30.72it/s]

✅ 완료: ./release/202402666.test.task1.txt에 결과 저장됨.
